In [ ]:
%config IPCompleter.greedy=True
!pip install azureml-pipeline-wrapper==0.1.0.13166565 --extra-index-url https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/13166565 --ignore-installed

In [1]:
from azureml.core import Workspace, Run, Dataset
from azureml.pipeline.wrapper import Pipeline, Module

ws = Workspace.get(name='jietong-test-westeurope', subscription_id='f8f8b81c-3ebd-427c-b3b7-ebb226c4eea2',
                   resource_group='studio-v2-test')

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [2]:
# Load modules from workspace
train_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Train')
score_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Score')
eval_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Evaluate')

# Load a module from local as anonymous module
#a_module_in_development_func = Module.load_from_yaml(ws, path='/home/code/new_module/spec.yaml')

train_data = Dataset.get_by_name(ws, 'training_data')
test_data = Dataset.get_by_name(ws, 'test_data')

In [ ]:
help(train_module_func)

In [3]:
train = train_module_func(
    training_data=train_data, 
    max_epochs=5, 
    learning_rate=0.02)

score = score_module_func(
    model_input=train.outputs.model_output, 
    test_data=test_data)

eval = eval_module_func(scoring_result=score.outputs.score_output)

pipeline = Pipeline(nodes=[train, score, eval], outputs=eval.outputs, name='a sample 3 step pipeline')

pipeline.validate(ws, default_compute_target='cpu-cluster')


WARNING - Datetime with no tzinfo will be considered UTC.
WARNING - Datetime with no tzinfo will be considered UTC.
WARNING - Datetime with no tzinfo will be considered UTC.


[{'error': []}, {'error': []}, {'error': []}]

In [ ]:
run = pipeline.submit_run(
    ws, 
    experiment_name='sample-pipelines', 
    default_compute_target='cpu-cluster')

run.wait_for_completion()